# Reduccion de variables


In [1]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [2]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from test_helper import Test

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

from sklearn import metrics, neighbors, tree, preprocessing
from scipy import stats

from matplotlib.colors import ListedColormap

from shutil import copyfile

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import neighbors, model_selection, tree
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)

# Se importa la librería de selección de variables
from sklearn import feature_selection
from sklearn.feature_selection import SelectPercentile
from sklearn.decomposition import PCA

### Datos

In [3]:
dataset = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
#se eliminan las filas que tienen valores nulos de cara a realizar posteriores estudios
#HABRIA QUE TENERLO IMPUTADO CON VALORES PERDIDOS!!
dataset.dropna(axis=0, inplace = True)

atSalida = 'close'
atEntrada = ['open','high','low','volume']

dataset = dataset[dataset['Name']=='AAL']
dataset_output = dataset[atSalida].copy()
dataset.drop(atSalida, axis=1, inplace=True)
dataset = dataset[atEntrada].copy()


# SUSTITUIR POR LO BUENO
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_output, test_size=0.2, random_state=12)

### Seleccion de variables

Utilizaremos diferentes tecnicas de seleccion de variables, compararemos sus errores y eligiremos la mejor.

* Correlación mutua: función [*f_regression*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)
* Información mutua: función [*mutual_info_regression*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression)


Una vez conocida la calidad de cada variable se deben escoger las mejores. Para ello vimos en las clases teóricas que había varias opciones. La librería Scikit-learn ofrece dos de estas técnicas en forma de clases (con sus campos y sus métodos):
* Elegir las k mejores con la clase [*SelectKBest*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)
* Elegir las variables en base al percentil (el % de las variables) con la clase [*SelectPercentile*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile)



 Correlación mutua: función [*f_regression*]

In [4]:
# Se crea la Pipeline con las fases deseadas de Correlación mutua
pipe_f_regression = Pipeline([
    ('correlacion_mutua', feature_selection.SelectPercentile(feature_selection.f_regression)),
    ('polynomial', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())])
# Se crea el grid de híper-parámetros
parameters_f_regression = {'correlacion_mutua__percentile': [10, 20, 30]}

* Información mutua: función [*mutual_info_regression*]

In [34]:
# Se crea la Pipeline con las fases deseadas de Informacion mutua
pipe_mutual_info_regression = Pipeline([
    ('polynomial', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('informacion_mutua', feature_selection.SelectPercentile(feature_selection.mutual_info_regression,percentile=10)),
    ('regressor', LinearRegression())])
# Se crea el grid de híper-parámetros
parameters_mutual_info_regression = {'informacion_mutua__percentile': [10, 20, 30]}

In [35]:
# Selecciona la pipe a utilizar
pipe = pipe_f_regression
parameters = parameters_f_regression

# Se llama al constructor de GridSearchCV para que genere todas las combinaciones de los híper-parámetros definidos anteriormente
gridSearch_pipe = model_selection.GridSearchCV(pipe,parameters,cv=3,return_train_score=True)
# Se realiza el aprendizaje de todos los clasificadores considerados (todas las configuraciones)
gridSearch_pipe = gridSearch_pipe.fit(X_train,y_train.values.ravel())
# Se muestra la mejor configuración junto con su rendimiento
print(gridSearch_pipe.best_score_)
print(gridSearch_pipe.best_params_)


# Almacenamos el DataFrame con los resultados
diccionarioResultados = gridSearch_pipe.cv_results_

# Se imprime el mejor porcentaje de acierto y los resultados de todas las configuraciones
print(gridSearch_pipe.best_score_,gridSearch_pipe.best_params_)

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
accTrain = gridSearch_pipe.score(X_train,y_train.values.ravel())*100
accTest = gridSearch_pipe.score(X_test,y_test.values.ravel())*100
print('Resultado en entrenamiento: {}%'.format(accTrain))
print('Resultado en test: {}%'.format(accTest))

0.9977987477718774
{'correlacion_mutua__percentile': 10}
0.9977987477718774 {'correlacion_mutua__percentile': 10}
Resultado en entrenamiento: 99.81172914050642%
Resultado en test: 99.71610456822759%


Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=0.1,
                     random_state=None, svd_solver='full', tol=0.0,
                     whiten=False)),
                ('polynomial',
                 PolynomialFeatures(degree=2, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regressor',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [8]:
pipe_mutual_info_regression = Pipeline([
    ('polynomial', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('informacion_mutua', feature_selection.SelectPercentile(feature_selection.mutual_info_regression,percentile=10)),
    ('regressor', LinearRegression())])
gridSearch_pipe = pipe_mutual_info_regression.fit(X_train,y_train.values.ravel())

PCA

In [9]:
# Se crea la Pipeline con las fases deseadas para el pipe PCA
pipe_pca = Pipeline([('pca', PCA(svd_solver='full')),
                 ('polynomial', PolynomialFeatures(degree=2)),
                 ('scaler', StandardScaler()),
                 ('regressor', LinearRegression())])
# Se crea el grid de híper-parámetros
parameters_pca = {'pca__n_components': [0.10, 0.30, 0.50, 0.70, 0.90]}
# Se llama al constructor de GridSearchCV para que genere todas las combinaciones de los híper-parámetros definidos anteriormente
gridSearch_pipe = model_selection.GridSearchCV(pipe_pca,parameters_pca,cv=10,n_jobs=-1,return_train_score=True)
# Se realiza el aprendizaje de todos los clasificadores considerados (todas las configuraciones)
gridSearch_pipe =  gridSearch_pipe.fit(X_train,y_train)
# Se muestra la mejor configuración junto con su rendimiento
print(gridSearch_pipe.best_score_)
print(gridSearch_pipe.best_params_)


# Almacenamos el DataFrame con los resultados
diccionarioResultados = gridSearch_pipe.cv_results_

# Se imprime el mejor porcentaje de acierto y los resultados de todas las configuraciones
print(gridSearch_pipe.best_score_,gridSearch_pipe.best_params_)

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
accTrain = gridSearch_pipe.score(X_train,y_train.values.ravel())*100
accTest = gridSearch_pipe.score(X_test,y_test.values.ravel())*100
print('Resultado en entrenamiento: {}%'.format(accTrain))
print('Resultado en test: {}%'.format(accTest))

-0.11193190724536053
{'pca__n_components': 0.1}
-0.11193190724536053 {'pca__n_components': 0.1}
Resultado en entrenamiento: 0.11252279355632444%
Resultado en test: -0.43812629193509245%


### Seleccióndeinstancias

Utilizamos Wrapers ya que miran la precision con Leave One Out y no tiene en cuanta las clases, las cuales no nos importan ennuestro modelo de regresion

In [31]:
def RMHC(regresor, X, y, s=0.1, iteraciones=1000):
    """
     Algoritmo RMHC (Random Mutation Hill Climbing) para la selección de instancias.
      Se comienza con una selección aleatoria de s * nEjemplos instancias. Para cada iteración, se elige una instancia
      seleccionada y una no seleccionada para ser intercambiadas. Si el intercambio mejora la precisión (leave-one-out) sobre train
      se mantiene el cambio, sino se deshace
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param s: Porcentaje de instancias a ser seleccionadas (s es un valor entre 0 y 1)
    :param iteraciones: Número de iteraciones (intercambios) a probar
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    print(X_train)
    # Como usamos aleatorios en RMHC establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
    np.random.seed(12312)

    # En este caso vamos a mantener dos vectores de enteros con los índices de las instancias seleccionadas y no seleccionadas
    # Establecemos el número de instancias a seleccionar (es fijo)
    nSel = int(X.shape[0]*s)
    
    # Obtenemos una permutación aleatoria de todos los índices disponibles 
    # (un array de 0 a X.shape[0] con valores ordenados aleatoriamente): utilizad permutation de Numpy
    permute = np.random.permutation(X.shape[0])
    
    # Cogemos como seleccionadas las instancias correspondientes a los primeros nSel índices  y no seleccionadas el resto#     
    seleccionadas = permute[:nSel]
    noSeleccionadas = permute[nSel:]
    print(seleccionadas)
    # Calculamos la preicisión con la selección inicial
    # Entrenamos regresor con las instancias seleccionadas
    regresor.fit(X[seleccionadas], y[seleccionadas])
    
    # Obtenemos las salidas con leaveOneOut (para no tener en cuenta las instancias seleccionadas como vecinos de sí mismas)
    salidas = regresor.predict(X)
        
    # Calculamos la precisión
    acc = metrics.accuracy_score(salidas, y)

    # Comenzamos las iteraciones:
    #   En cada una, intercambiamos una instancia seleccionada con una que no lo está y comprobamos si mejora la precisión
    for i in range(0, iteraciones):
        # Seleccionamos un índice de manera aleatoria que nos indica la instancia a eliminar de las seleccionadas: randint de Numpy
        quitar = np.random.randint(seleccionadas.size)
        
        # Seleccionamos un índice de manera aleatoria que nos indica la instancia a añadir a las seleccionadas: randint de Numpy
        poner = np.random.randint(noSeleccionadas.size)
        
        # Guardamos el ejemplo eliminado
        aux =  seleccionadas[quitar]
        
        # Añadimos la nueva instancia seleccionada sustituyendo la que se elimina: intercambio en las listas de seleccionados y no seleccionados
        seleccionadas[quitar] = noSeleccionadas[poner]
        
        # Entrenamos de nuevo regresor
        regresor.fit(X[seleccionadas], y[seleccionadas])

        # Calculamos la precisión de la solución actual
        # Obtenemos las salidas con leaveOneOut (para no tener en cuenta las instancias seleccionadas como vecinos de sí mismas)
        salidas = regresor.predict(X)
        
        # Calculamos la precisión
        accNew = metrics.accuracy_score(salidas, y)

        # Si la precisión actual es peor que la anterior, devolvemos la instancia eliminada a las seleccionadas
        # Sino si la precisión de la solución actual es mejor o igual que la anterior, guardamos la precisión y
        #  añadimos la instancia eliminada a las no seleccionadas
        if accNew >= acc:
            acc = accNew
            noSeleccionadas[poner] = aux
        else:
            seleccionadas[quitar] = aux

        if i % 100 == 0:
            print("precision en iteracion {}: {}".format(i, acc))

    # A partir de las seleccionadas creamos la máscara de instancias seleccionadas 
    # donde True en la posición i indica que la instancia i es seleccionada
    S = np.zeros(len(y), bool)
    S[seleccionadas] = True

    return S

In [32]:
('polynomial', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('informacion_mutua', feature_selection.SelectPercentile(feature_selection.mutual_info_regression,percentile=10)),
    ('regressor', LinearRegression())])

array([[1.7860000e+01, 1.8000000e+01, 1.7750000e+01, 3.8391000e+06],
       [3.5780000e+01, 3.6080000e+01, 3.5330000e+01, 7.9562710e+06],
       [1.8790000e+01, 1.9280000e+01, 1.8680000e+01, 5.6109000e+06],
       ...,
       [1.5990000e+01, 1.6000000e+01, 1.5500000e+01, 9.2271000e+06],
       [3.7270000e+01, 3.7630000e+01, 3.6680000e+01, 3.8630640e+06],
       [3.9600000e+01, 4.0400000e+01, 3.9290000e+01, 1.3355207e+07]])

In [50]:
poly = PolynomialFeatures(2,include_bias=False)
X_train_poly = poly.fit_transform(X_train.values)

scaler = StandardScaler()
X_train_poly_scaler = scaler.fit_transform(X_train)

sp = SelectPercentile(feature_selection.mutual_info_regression, percentile=10)
X_train_poly_scaler_sp = sp.fit_transform(X_train_poly_scaler, y_train)


In [49]:
X_train_poly_scaler_sp

array([[-1.87705818, -1.68294735],
       [-0.24641643, -0.42231598],
       [-1.76161451, -1.62142416],
       ...,
       [-2.0574389 , -1.77061955],
       [-0.10662137, -0.27501315],
       [ 0.14320593,  0.00365952]])

In [51]:
S = RMHC(LinearRegression(), X_train_poly_scaler_sp, y_train)

      open   high    low    volume
108  17.86  18.00  17.75   3839100
866  35.78  36.08  35.33   7956271
157  18.79  19.28  18.68   5610900
9    13.57  13.60  13.21   6071400
134  15.80  15.99  15.75   4234300
..     ...    ...    ...       ...
418  34.75  35.02  34.06  10032531
473  50.66  51.69  50.35   5961395
49   15.99  16.00  15.50   9227100
278  37.27  37.63  36.68   3863064
432  39.60  40.40  39.29  13355207

[1007 rows x 4 columns]
[134 791 583 167 200 670 373 129 674  80 315 788 613 894 343 644 944 236
 303 415  19 400 734 862 166  30 619 906 968 871 387 604 863 512 810 124
 188 897 492 139 123 861 507 695 848 623 130 111 694 718  64 622 156 931
 859 237 185 802 221 955 210 852 666 216  50 570 202 685 133 652 494 972
 820 815 713 823 697 280 501 390 625 910  63 768 131 830 227 241 720 811
 665 786 466 984  31 224 678 598 945 748]


C:\Users\daviz.DESKTOP-829TKAF\anaconda3\envs\py37machlearn\lib\site-packages\pandas\core\series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
print(X_train.shape[0])
print(S.shape[0])

### PROPHET FACEBOOK IMPLEMENTATION 

In [ ]:
pip install fbprophet

In [ ]:
from fbprophet import Prophet

Dataset solo tiene las variables 'date' y 'close' (o la ue se quiera predecir)

In [ ]:
dataset=dataset_valor_anterior.copy()
dataset = dataset[dataset.Name=="AAPL"]
atEntrada = ['date','close']

dataset = dataset[atEntrada].copy()
df = dataset.rename(columns = {'date':'ds','close': 'y'}, inplace = False)
df

In [ ]:
m = Prophet()
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=365)
# print(future.tail())
forecast = m.predict(future)
forecast.tail()
# forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()
# fig1 = m.plot(forecast)
# m.plot_components(forecast)


In [ ]:
m.plot_components(forecast)